In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
from sqlalchemy import (create_engine, MetaData, Table, Column, String, Numeric, Date)
from datetime import datetime

# Extract and Transform

In [2]:
firms = ["KMB","KO","PM","MGNT.ME","FIVE.ME"]

In [3]:
def stock_price_hist(firms):
    """Getting history of the stock price in USD"""
    dfs = []
    for s in firms:
        stocks = yf.Ticker(s)
        hist = stocks.history(period="5y")
        hist.reset_index(inplace=True)
        df = hist.loc[hist.Date >= "2018-01-01",["Date","Close","Dividends"]]
        df["Stock"] = s
        dfs.append(df)
        all_firms = pd.concat(dfs)
        all_firms.reset_index(inplace=True, drop=True)
        mask1 = all_firms.Stock.str.lower().str.contains(".me")
        all_firms.loc[mask1,"Close"] = all_firms["Close"] / 70
        mask2 = all_firms.Stock.str.lower().str.contains("mgnt")
        all_firms.loc[mask2,["Dividends"]] = (all_firms["Dividends"] / 70)
    return all_firms

In [4]:
df = stock_price_hist(firms)

In [5]:
def financial_indicators(firms):
    """Getting info on Total Revenue, Net Income, Cash flow etc."""
    dfs = []
    for s in firms:
        stocks = yf.Ticker(s)
        f = stocks.financials
        finance = f.loc[["Total Revenue","Net Income"],:].copy()
        b = stocks.balance_sheet
        balance = b.loc[["Cash","Total Assets","Short Long Term Debt","Long Term Debt"],:].copy()
        c = stocks.cashflow
        cashflow = c.loc[["Capital Expenditures"],:].copy()
        df = pd.concat([finance,balance,cashflow])
        dft = df.T
        dft["Stock"] = s
        dfs.append(dft)
        mixed_firms = pd.concat(dfs)
        mixed_firms.reset_index(inplace=True)
        mixed_firms["Date"] = mixed_firms.iloc[:,0]
        mixed_firms.drop(mixed_firms.columns[0], axis=1, inplace=True)
        df_us = mixed_firms.loc[~mixed_firms.Stock.str.lower().str.contains(".me")].copy()
        df_rus = mixed_firms.loc[mixed_firms.Stock.str.lower().str.contains(".me")].copy()
        mask = df_rus.iloc[:,1:5]
        df_rus.iloc[:,1:5] = mask / 70
        df_all = pd.concat([df_us, df_rus])
    return df_all

In [6]:
df1 = financial_indicators(firms)

In [7]:
df.rename(columns={
                    "Date":"date",
                    "Close":"close",
                    "Dividends":"dividends",
                    "Stock":"stock"}, inplace=True)

df1.rename(columns={"Total Revenue":"total_rev",
                    "Net Income":"n_income",
                    "Cash":"cash",
                    "Total Assets":"assets",
                    "Short Long Term Debt":"s_l_t_dept",
                    "Long Term Debt":"l_t_dept",
                    "Capital Expenditures":"cap_expend",
                    "Stock":"stock",
                    "Date":"date"}, inplace=True)

# Load

In [8]:
meta_data = MetaData(schema="dashboarding")
db_connection = 'postgresql://postgres:<PASSWORD>@localhost:5432/postgres'

In [9]:
stock_price_hist = Table('stock_price_hist',meta_data,
                        Column('date', Date,nullable=False),
                        Column('close', Numeric(),nullable=False),
                        Column('dividends', Numeric(),nullable=False),
                        Column('stock', String(20),nullable=False)
                        )


financial_indicators = Table('financial_indicators',meta_data,
                        Column('total_rev', Numeric(),nullable=True),
                        Column('n_income', Numeric(),nullable=True),
                        Column('cash', Numeric(),nullable=True),
                        Column('assets', Numeric(),nullable=True),
                        Column('s_l_t_dept', Numeric(),nullable=True),
                        Column('l_t_dept', Numeric(),nullable=True),
                        Column('cap_expend', Numeric(),nullable=True),
                        Column('stock', String(20),nullable=True),
                        Column('date', Date,nullable=True)
                        )

In [10]:
engine = create_engine(db_connection)

try:
  conn = engine.connect()
  print('db connected')
  print('connection object is :{}'.format(conn))
except: 
  print('db not connected')

meta_data.create_all(engine)

db connected
connection object is :<sqlalchemy.engine.base.Connection object at 0x7fdba267ed30>


In [11]:
try:    
    df1.to_sql('financial_indicators', con=engine, index=False, if_exists='append', schema='dashboarding', method='multi')
    print("Data is successfully loaded")
except:
    print("Data cannot be loaded")

Data is successfully loaded


In [12]:
try:    
    df.to_sql('stock_price_hist', con=engine, index=False, if_exists='append', schema='dashboarding',method='multi')
    print("Data is successfully loaded")
except:
    print("Data cannot be loaded")

Data is successfully loaded
